# VideoMAE / 15 Frame / 224 224 3 / R2plu1d Backbone / masking ratio = 0.75 / embed_dim = 96

In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
import logging
import torch
import torch.nn as nn
from einops import rearrange, reduce
from tqdm import tqdm
import gc
import copy

# 사용하고자 하는 model
from models_mae_R2plus1D import MaskedAutoencoderViT

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/experiments11.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)

In [2]:
batchsz = 8
num_workers = 4
epochs = 1000
start_epoch = 0
lr = 3e-5

In [3]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:1' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

True
학습을 진행하는 기기: cuda:1


In [4]:
with open('/home/oem/kw/개성형성/BackgroundRandomRemove/Dataset/15frame/training(FullShot_15Frame).dat', "rb") as training_file:
    train_input_temp = pickle.load(training_file)
with open('/home/oem/kw/개성형성/BackgroundRandomRemove/Dataset/15frame/validation(FullShot_15Frame).dat', "rb") as validation_file:
    valid_input_temp = pickle.load(validation_file)

In [5]:
print(train_input_temp[0].shape)
print(valid_input_temp[0].shape)

(5979, 15, 224, 224, 3)
(1993, 15, 224, 224, 3)


In [6]:
class ChalearnDataset(Dataset):
    def __init__(self, fullshot, label):
        self.fullshot = fullshot
        self.label = label
        
    def __len__(self):
        return len(self.fullshot)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        fullshot_data = self.fullshot[idx]
        fullshot_data = torch.FloatTensor(fullshot_data)

        label_data = self.label[idx]
        label_data = torch.FloatTensor(label_data)
        
        return fullshot_data, label_data

In [7]:
train_dataset = ChalearnDataset(fullshot = train_input_temp[0], label = train_input_temp[1])
val_dataset = ChalearnDataset(fullshot = valid_input_temp[0], label = valid_input_temp[1])

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsz, shuffle=False, num_workers=num_workers)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsz, shuffle=False, num_workers=num_workers)

max_value=0

In [8]:
model=MaskedAutoencoderViT(num_frames = 4, t_patch_size = 2, img_size = 28, patch_size = 4, embed_dim = 96)

model.to(device)

criterion = torch.nn.L1Loss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

R2plus1d_pretrained_model
img_size (28, 28) patch_size (4, 4) frames 4 t_patch_size 2


In [9]:
from torchinfo import summary

summary(model, input_size = (16,3,15,224,224), col_names = ['input_size','output_size','num_params'])

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #
MaskedAutoencoderViT                               [16, 3, 15, 224, 224]     [16, 5]                   60,096
├─VideoResNet: 1-1                                 [16, 3, 15, 224, 224]     [16, 256, 4, 28, 28]      --
│    └─R2Plus1dStem: 2-1                           [16, 3, 15, 224, 224]     [16, 64, 15, 112, 112]    --
│    │    └─Conv3d: 3-1                            [16, 3, 15, 224, 224]     [16, 45, 15, 112, 112]    6,615
│    │    └─BatchNorm3d: 3-2                       [16, 45, 15, 112, 112]    [16, 45, 15, 112, 112]    90
│    │    └─ReLU: 3-3                              [16, 45, 15, 112, 112]    [16, 45, 15, 112, 112]    --
│    │    └─Conv3d: 3-4                            [16, 45, 15, 112, 112]    [16, 64, 15, 112, 112]    8,640
│    │    └─BatchNorm3d: 3-5                       [16, 64, 15, 112, 112]    [16, 64, 15, 112, 112]    128
│    │    └─ReLU: 3-6         

In [10]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

11336062

In [11]:
sum(p.numel() for p in model.parameters())

11336062

In [12]:
torch.set_printoptions(profile="full")

trainingEpoch_loss = []
validationEpoch_loss = []

for i in range(start_epoch,epochs):
    train_avg_loss = 0
    val_avg_loss = 0
    
    model.train()
    for fullshot_data, label_data in tqdm(train_dataloader):
        if len(fullshot_data) != batchsz:
            continue
        # full shot
        fullshot_data = rearrange(fullshot_data, 'b d h w c -> b c d h w')
        fullshot_data = fullshot_data.to(device)

        # label
        label_data = reduce(label_data,'b c d -> b c', 'max')
        label_data = label_data.to(device)
        
        # 학습
        optimizer.zero_grad()
        hypothesis = model(fullshot_data)

        loss = criterion(hypothesis, label_data)
        
        loss.backward()
        optimizer.step()
        
        train_avg_loss += loss.item()
    train_avg_loss=train_avg_loss/len(train_dataloader)
    trainingEpoch_loss.append(train_avg_loss)
    print('Epoch = {}, 1 - train_loss = {}'.format(i+1,(1 - train_avg_loss)))
    logger.debug('Epoch = {}, 1 - train_loss = {}'.format(i+1,(1 - train_avg_loss)))

    
    
    with torch.no_grad():
        model.eval()
        for fullshot_data, label_data in tqdm(val_dataloader):
            if len(fullshot_data) != batchsz:
                continue
            
            # full shot
            fullshot_data = rearrange(fullshot_data, 'b d h w c -> b c d h w')
            fullshot_data = fullshot_data.to(device)

            # label
            label_data = reduce(label_data,'b c d -> b c', 'max')
            label_data = label_data.to(device)
                
            hypothesis = model(fullshot_data)
            
            val_loss = criterion(hypothesis, label_data)
            val_avg_loss += val_loss.item()
            
        val_avg_loss=val_avg_loss/len(val_dataloader)
        validationEpoch_loss.append(val_avg_loss)
    print('Epoch = {}, 1 - val_loss = {}'.format(i+1,(1 - val_avg_loss)))
    logger.debug('Epoch = {}, 1 - valid_loss = {}'.format(i+1,(1-val_avg_loss)))
    print('\n')

    start_epoch+=1
    # if i%50 == 0:        
    #     torch.save({
    #         'epoch': i+1,
    #         'model': model.state_dict(),
    #         'optimizer': optimizer.state_dict(),
    #     }, save_model_file_path.format('model',start_epoch,'pth'))
    
    torch.cuda.empty_cache()
    gc.collect()

  0%|          | 0/748 [00:00<?, ?it/s]

100%|██████████| 748/748 [05:16<00:00,  2.36it/s]
[2024-03-12 22:40:10,480]::269792397::DEBUG::Epoch = 1, 1 - train_loss = 0.4250172457513325


Epoch = 1, 1 - train_loss = 0.4250172457513325


100%|██████████| 250/250 [00:34<00:00,  7.34it/s]
[2024-03-12 22:40:44,562]::269792397::DEBUG::Epoch = 1, 1 - valid_loss = 0.8748489592075348


Epoch = 1, 1 - val_loss = 0.8748489592075348




...

100%|██████████| 748/748 [05:17<00:00,  2.36it/s]
[2024-03-13 12:14:39,866]::269792397::DEBUG::Epoch = 140, 1 - train_loss = 0.9605070000071139


Epoch = 140, 1 - train_loss = 0.9605070000071139


100%|██████████| 250/250 [00:34<00:00,  7.31it/s]
[2024-03-13 12:15:14,091]::269792397::DEBUG::Epoch = 140, 1 - valid_loss = 0.913123510286212


Epoch = 140, 1 - val_loss = 0.913123510286212




 85%|████████▌ | 639/748 [04:31<00:45,  2.37it/s]